## View example downloaded EMIT agricultural scenes

We will use the image granules that you ortho-mosaiced in the previous step **"2_Orthorectify_images.ipynb"**

### Step 1. Setup notebook

Import packages

In [ ]:
import os, sys, fnmatch
import warnings
import csv
from osgeo import gdal
import numpy as np
import math
import rasterio as rio
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc

# This will ignore some warnings caused by holoviews
warnings.simplefilter('ignore') 

### Step 2. Find all downloaded and ortho-rectified agricultural images

In [ ]:
# Define workflow which selects the appropriate image data folder
workflow = "agriculture"
source_file_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data","emit",workflow)

In [ ]:
granules = fnmatch.filter(os.listdir(source_file_path), '*ortho.nc')
granules

### Step 3. Select and load a previously orthorectified EMIT image

In [ ]:
# Pick an image granule to explore
#img_file = 'EMIT_L2A_RFL_001_20230729T205642_2321014_020.nc'
img_file = 'EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc'

In [ ]:
# # Load the selected image to memory
img_file_dat = os.path.join(source_file_path,img_file)
ds_geo = xr.open_dataset(img_file_dat)
ds_geo

### Step 4. Quickly display the selected ortho-rectified image

In [ ]:
refl850 = ds_geo.sel(wavelengths=850, method='nearest')
ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl850.wavelengths.values:.3f} {refl850.wavelengths.units} (Orthorectified)")

We can see that the orthorectification step placed the data on a geogrpahic grid that matches pretty well with ESRI tiles. Now that we have a better idea of what the target area looks like, we can also plot the spectra using the georeferenced data. 

Similarly, we can review a wavelength in the visible spectrum.  Let's use a wavelength found within the green wavelengths 

In [ ]:
refl550 = ds_geo.sel(wavelengths=550, method='nearest')
ds_geo.sel(wavelengths=550, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl550.wavelengths.values:.3f} {refl550.wavelengths.units} (Orthorectified)")

In [ ]:
refl650 = ds_geo.sel(wavelengths=650, method='nearest')
ds_geo.sel(wavelengths=650, method='nearest').hvplot.image(cmap='Viridis', geo=True, tiles='ESRI', alpha=0.8, frame_height=600).opts(
    title=f"Reflectance at {refl650.wavelengths.values:.3f} {refl650.wavelengths.units} (Orthorectified)")

We can also display all three bands side-by-side to look a the differences in reflectance at different wavelengths from the visible to the near-infrared

In [ ]:
refl550.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 550") + \
refl650.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 650") + \
refl850.hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500).opts(title="Band: 850")

### Step 5. Plot example spectra

Now let's plot some example spectra found in the image. Before we do this, we should filter out the water absorption bands like we did earlier. By limiting the third dimension of the array to good_wavelengths.

In [ ]:
ds_geo['reflectance'].data[:,:,ds_geo['good_wavelengths'].data==0] = np.nan

In [ ]:

point = ds_geo.sel(longitude=-102.694,latitude=40.347,method='nearest')
point.hvplot.line(y='reflectance',x='wavelengths', color='black', frame_height=400, frame_width=600).opts(
    title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')

### Step 6. Experiment with band ratios or what are called Specrtal Vegetation Indices (SVIs)

To explore the utility of high spectral resolution data for calculating SVIs for Earth Science, we can demonstrate how to caluclate a commonly-used index: The Normalized Difference Vegetation Index (NDVI).  NDVI has been used for over 40 years to study changes on the Earth's surface, specifically related to vegetation, stress, and agriculture. For more information on NDVI, you can explore this article from NASA: [https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php)

In addition, here is a recent article describing the use of SVIs in Earth Science <br>
[https://www.nature.com/articles/s41597-023-02096-0](https://www.nature.com/articles/s41597-023-02096-0)

Calculate the Normalized Difference Vegetation Index (NDVI)

In [ ]:
# NDVI uses a combination of reflectance in the NIR and Red wavelengths.  For example, 850 and 655 nm
# First, let's calculate an NDVI image and take a look at the results
refl650 = ds.sel(wavelengths=650, method='nearest')
refl850 = ds.sel(wavelengths=850, method='nearest')
ndvi = (refl850-refl650)/(refl850+refl650)

In [ ]:
ndvi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

In [ ]:
### STILL A WIP!!!